# Evaluation of effect of increasing epsilon with CMC-learners and other approaches (simulations)

**On Nie and Wager (2021) synthetic data**

In [1]:
# import everyting from data module which is located ../src/data.py
import sys
import numpy as np
sys.path.append('../..')
from src.datasets.r_learner_datasets import (simulate_nuisance_and_easy_treatment,
                                             simulate_randomized_trial,
                                             simulate_easy_propensity_difficult_baseline,
                                             simulate_unrelated_treatment_control)

In [2]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

NSim = 20
epsilons = np.arange(0, 1.1, 0.1)
learner = RandomForestRegressor
learner_name = "RF"
MC_samples = 100
adaptive_conformal = True
if adaptive_conformal:
    adaptive_conformal_name = "adaptive"
else:
    adaptive_conformal_name = "nonadaptive"

In [ ]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from src.mc_conformal_metalearner.mc_conformal_metalearners import Conformal_MC_S_Learner, Conformal_MC_T_Learner, Conformal_MC_X_Learner
from src.conformal_metalearners.CM_learner import CM_learner
from tqdm import tqdm

from src.weighted_conformal_prediction.wcp import NaiveWCP, NestedWCP


for i, setup in enumerate([simulate_nuisance_and_easy_treatment, simulate_randomized_trial, simulate_easy_propensity_difficult_baseline, simulate_unrelated_treatment_control]):
    if i == 0:
        setup_name = "A"
        continue
    elif i == 1:
        setup_name = "B"
        continue
    elif i == 2:
        setup_name = "C"
    else:
        setup_name = "D"

    for n in tqdm(range(NSim)):
        coverage_y0 = []
        coverage_y1 = []
        coverage_pseudo_MC_T_ite = []
        coverage_MC_T_ite = []
        coverage_pseudo_MC_S_ite = []
        coverage_MC_S_ite = []
        coverage_pseudo_MC_X_ite = []
        coverage_MC_X_ite = []
        coverage_CM_ite = []
        coverage_naive_ite = []
        coverage_exact_ite = []
        coverage_inexact_ite = []

        int_width_y0 = []
        int_width_y1 = []
        int_width_pseudo_MC_T = []
        int_width_MC_T = []
        int_width_pseudo_MC_S = []
        int_width_MC_S = []
        int_width_pseudo_MC_X = []
        int_width_MC_X = []
        int_width_CM = []
        int_width_naive_ite = []
        int_width_exact_ite = []
        int_width_inexact_ite = []

        rmse_y0 = []
        rmse_y1 = []
        rmse_pseudo_MC_T_ite, rmse_pseudo_MC_T_ite_conformal_mean = [], []
        rmse_MC_T_ite,  rmse_MC_T_ite_conformal_mean = [], []
        rmse_pseudo_MC_S_ite, rmse_pseudo_MC_S_ite_conformal_mean = [], []
        rmse_MC_S_ite, rmse_MC_S_ite_conformal_mean = [], []
        rmse_pseudo_MC_X_ite, rmse_pseudo_MC_X_ite_conformal_mean = [], []
        rmse_MC_X_ite, rmse_MC_X_ite_conformal_mean = [], []
        rmse_CM_ite = []
        rmse_naive_ite = []
        rmse_exact_ite = []
        rmse_inexact_ite = []
        for epsilon in tqdm(epsilons):
            y, X, W, tau, b, e, y0, y1 = setup(n=5000, c=epsilon)
            ite = y1 - y0
            ps = e
            (y_train, y_test, X_train, X_test,
            W_train, W_test,
            tau_train, tau_test, b_train, b_test,
            ps_train, ps_test, y0_train, y0_test,
            y1_train, y1_test, ite_train, ite_test) = train_test_split(y, X, W, tau, b, ps, y0, y1, ite, test_size=0.5)
            # df_X_train = pd.DataFrame(X_train, columns=[f"X{i+1}" for i in range(X_train.shape[1])])
            # df_X_test = pd.DataFrame(X_test, columns=[f"X{i+1}" for i in range(X_test.shape[1])])
            # df_other_train = pd.DataFrame({"y":y_train, "W":W_train, "tau":tau_train, "b":b_train, "ps":ps_train, "y0":y0_train, "y1":y1_train, "ite":ite_train})
            # df_other_test = pd.DataFrame({"y":y_test, "W":W_test, "tau":tau_test, "b":b_test, "ps":ps_test, "y0":y0_test, "y1":y1_test, "ite":ite_test})
            # df_train = pd.concat([df_X_train, df_other_train], axis=1)
            # df_test = pd.concat([df_X_test, df_other_test], axis=1)
            # df_train.to_csv(f"../results/outputs/r_learner/setup{setup_name}/simulations_{setup_name}_{str(n)}_epsilon_{int(epsilon*100)}_train.csv")
            # df_test.to_csv(f"../results/outputs/r_learner/setup{setup_name}/simulations_{setup_name}_{str(n)}_epsilon_{int(epsilon*100)}_test.csv")
            # Initialize the learner
            conformal_pseudo_MC_T_Learner = Conformal_MC_T_Learner(
                learner(),
                learner(),
                adaptive_conformal=adaptive_conformal,
                pseudo_MC=True,
                MC_samples=100,
            )

            conformal_MC_T_Learner = Conformal_MC_T_Learner(
                learner(),
                learner(),
                adaptive_conformal=adaptive_conformal,
                pseudo_MC=False,
                MC_samples=100,
            )
            conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train)
            conformal_MC_T_Learner.fit(X_train, y_train, W_train)
            conformal_pseudo_MC_S_Learner = Conformal_MC_S_Learner(
                        learner(),
                        adaptive_conformal=adaptive_conformal,
                        pseudo_MC=True,
                        MC_samples=100,
                    )
            with warnings.catch_warnings(action="ignore", category=UserWarning):
                # Suppress warning that is thrown saying that calibration example is too small
                # However, this is a bug in the crepes library in this case
                conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train)
            conformal_MC_S_Learner = Conformal_MC_S_Learner(
                learner(),
                adaptive_conformal=adaptive_conformal,
                pseudo_MC=False,
                MC_samples=100,
            )
            with warnings.catch_warnings(action="ignore", category=UserWarning):
                # Suppress warning that is thrown saying that calibration example is too small
                # However, this is a bug in the crepes library in this case
                conformal_MC_S_Learner.fit(X_train, y_train, W_train)
            conformal_pseudo_MC_X_Learner = Conformal_MC_X_Learner(
                learner(),
                learner(),
                learner(),
                adaptive_conformal=adaptive_conformal,
                pseudo_MC=True,
                MC_samples=100,
            )
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train)
            conformal_MC_X_Learner = Conformal_MC_X_Learner(
                learner(),
                learner(),
                learner(),
                adaptive_conformal=adaptive_conformal,
                pseudo_MC=False,
                MC_samples=100,
            )
            # Fit the learner
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                conformal_MC_X_Learner.fit(X_train, y_train, W_train)
            naive_WCP = NaiveWCP(
                    learner(),
                    learner(),
                    adaptive_conformal=adaptive_conformal)
            naive_WCP.fit(X_train, y_train, W_train, ps_train)



            alpha = 0.1

            int_y0_test = conformal_MC_T_Learner.predict_int_y0(X_test, confidence=1-alpha)
            int_y1_test = conformal_MC_T_Learner.predict_int_y1(X_test, confidence=1-alpha)
            int_pseudo_MC_ite_test = conformal_pseudo_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_ite_test = conformal_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_S_ite_test = conformal_pseudo_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_S_ite_test = conformal_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_X_ite_test = conformal_pseudo_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_X_ite_test = conformal_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            conformal_Learner = CM_learner(metalearner="DR", alpha=alpha)
            conformal_Learner.fit(X_train, y_train, W_train, ps_train)
            int_CM_ite_test = conformal_Learner.predict_int(X_test)
            int_naive_ite_test = naive_WCP.predict_int(X_test, ps_test, confidence=1-alpha)
            exact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            learner(),
                            adaptive_conformal=adaptive_conformal,
                            exact=True)
            exact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_exact_ite_test = exact_wcp.predict_int(X_test, ps_test)
            inexact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            adaptive_conformal=adaptive_conformal,
                            exact=False)
            inexact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_inexact_ite_test = inexact_wcp.predict_int(X_test, ps_test)
            coverage_y0.append(np.mean((int_y0_test[:, 0] < y0_test) &  (int_y0_test[:, 1] > y0_test)))
            coverage_y1.append(np.mean((int_y1_test[:, 0] < y1_test) &  (int_y1_test[:, 1] > y1_test)))
            coverage_pseudo_MC_T_ite.append(np.mean((int_pseudo_MC_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_ite_test[:, 1] > ite_test)))
            coverage_MC_T_ite.append(np.mean((int_MC_ite_test[:, 0] < ite_test) &  (int_MC_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_S_ite.append(np.mean((int_pseudo_MC_S_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_S_ite_test[:, 1] > ite_test)))
            coverage_MC_S_ite.append(np.mean((int_MC_S_ite_test[:, 0] < ite_test) &  (int_MC_S_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_X_ite.append(np.mean((int_pseudo_MC_X_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_X_ite_test[:, 1] > ite_test)))
            coverage_MC_X_ite.append(np.mean((int_MC_X_ite_test[:, 0] < ite_test) &  (int_MC_X_ite_test[:, 1] > ite_test)))
            coverage_CM_ite.append(np.mean((int_CM_ite_test[:, 0] < ite_test) &  (int_CM_ite_test[:, 1] > ite_test)))
            coverage_naive_ite.append(np.mean((int_naive_ite_test[:, 0] < ite_test) &  (int_naive_ite_test[:, 1] > ite_test)))
            coverage_exact_ite.append(np.mean((int_exact_ite_test[:, 0] < ite_test) &  (int_exact_ite_test[:, 1] > ite_test)))
            coverage_inexact_ite.append(np.mean((int_inexact_ite_test[:, 0] < ite_test) &  (int_inexact_ite_test[:, 1] > ite_test)))

            int_width_y0.append(np.diff(int_y0_test).mean())
            int_width_y1.append(np.diff(int_y1_test).mean())
            int_width_pseudo_MC_T.append(np.diff(int_pseudo_MC_ite_test).mean())
            int_width_MC_T.append(np.diff(int_MC_ite_test).mean())
            int_width_pseudo_MC_S.append(np.diff(int_pseudo_MC_S_ite_test).mean())
            int_width_MC_S.append(np.diff(int_MC_S_ite_test).mean())
            int_width_pseudo_MC_X.append(np.diff(int_pseudo_MC_X_ite_test).mean())
            int_width_MC_X.append(np.diff(int_MC_X_ite_test).mean())
            int_width_CM.append(np.diff(int_CM_ite_test).mean())
            int_width_naive_ite.append(np.diff(int_naive_ite_test).mean())
            int_width_exact_ite.append(np.diff(int_exact_ite_test).mean())
            int_width_inexact_ite.append(np.diff(int_inexact_ite_test).mean())

            rmse_y0.append(np.sqrt(np.mean((y0_test - conformal_MC_T_Learner.predict_y0(X_test))**2)))
            rmse_y1.append(np.sqrt(np.mean((y1_test - conformal_MC_T_Learner.predict_y1(X_test))**2)))
            rmse_pseudo_MC_T_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_T_Learner.predict(X_test))**2)))
            rmse_MC_T_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_T_Learner.predict(X_test))**2)))
            rmse_pseudo_MC_S_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_S_Learner.predict(X_test))**2)))
            rmse_MC_S_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_S_Learner.predict(X_test))**2)))
            rmse_pseudo_MC_X_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_X_Learner.predict(X_test))**2)))
            rmse_MC_X_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_X_Learner.predict(X_test))**2)))
            rmse_CM_ite.append(np.sqrt(np.mean((ite_test - conformal_Learner.predict(X_test))**2)))
            rmse_naive_ite.append(np.sqrt(np.mean((ite_test - naive_WCP.predict(X_test))**2)))
            rmse_exact_ite.append(np.sqrt(np.mean((ite_test - exact_wcp.predict(X_test))**2)))
            rmse_inexact_ite.append(np.sqrt(np.mean((ite_test - inexact_wcp.predict(X_test))**2)))

            # conformal_mean
            rmse_pseudo_MC_T_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_T_Learner.predict(X_test, conformal_mean=True))**2)))
            rmse_MC_T_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_MC_T_Learner.predict(X_test, conformal_mean=True))**2)))
            rmse_pseudo_MC_S_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_S_Learner.predict(X_test, conformal_mean=True))**2)))
            rmse_MC_S_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_MC_S_Learner.predict(X_test, conformal_mean=True))**2)))
            rmse_pseudo_MC_X_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_X_Learner.predict(X_test, conformal_mean=True))**2)))
            rmse_MC_X_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - conformal_MC_X_Learner.predict(X_test, conformal_mean=True))**2)))
        df_eval = pd.DataFrame({
            "coverage_y0": coverage_y0,
            "coverage_y1": coverage_y1,
            "coverage_pseudo_MC_T_ite": coverage_pseudo_MC_T_ite,
            "coverage_MC_T_ite": coverage_MC_T_ite,
            "coverage_pseudo_MC_S_ite": coverage_pseudo_MC_S_ite,
            "coverage_MC_S_ite": coverage_MC_S_ite,
            "coverage_pseudo_MC_X_ite": coverage_pseudo_MC_X_ite,
            "coverage_MC_X_ite": coverage_MC_X_ite,
            "coverage_CM_ite": coverage_CM_ite,
            "coverage_naive_ite": coverage_naive_ite,
            "coverage_exact_ite": coverage_exact_ite,
            "coverage_inexact_ite": coverage_inexact_ite,
            "int_width_y0": int_width_y0,
            "int_width_y1": int_width_y1,
            "int_width_pseudo_MC_T": int_width_pseudo_MC_T,
            "int_width_MC_T": int_width_MC_T,
            "int_width_pseudo_MC_S": int_width_pseudo_MC_S,
            "int_width_MC_S": int_width_MC_S,
            "int_width_pseudo_MC_X": int_width_pseudo_MC_X,
            "int_width_MC_X": int_width_MC_X,
            "int_width_CM": int_width_CM,
            "int_width_naive_ite": int_width_naive_ite,
            "int_width_exact_ite": int_width_exact_ite,
            "int_width_inexact_ite": int_width_inexact_ite,
            "rmse_y0": rmse_y0,
            "rmse_y1": rmse_y1,
            "rmse_pseudo_MC_T_ite": rmse_pseudo_MC_T_ite,
            "rmse_MC_T_ite": rmse_MC_T_ite,
            "rmse_pseudo_MC_S_ite": rmse_pseudo_MC_S_ite,
            "rmse_MC_S_ite": rmse_MC_S_ite,
            "rmse_pseudo_MC_X_ite": rmse_pseudo_MC_X_ite,
            "rmse_MC_X_ite": rmse_MC_X_ite,
            "rmse_CM_ite": rmse_CM_ite,
            "rmse_naive_ite": rmse_naive_ite,
            "rmse_exact_ite": rmse_exact_ite,
            "rmse_inexact_ite": rmse_inexact_ite,
            "rmse_pseudo_MC_T_ite_conformal_mean": rmse_pseudo_MC_T_ite_conformal_mean,
            "rmse_MC_T_ite_conformal_mean": rmse_MC_T_ite_conformal_mean,
            "rmse_pseudo_MC_S_ite_conformal_mean": rmse_pseudo_MC_S_ite_conformal_mean,
            "rmse_MC_S_ite_conformal_mean": rmse_MC_S_ite_conformal_mean,
            "rmse_pseudo_MC_X_ite_conformal_mean": rmse_pseudo_MC_X_ite_conformal_mean,
            "rmse_MC_X_ite_conformal_mean": rmse_MC_X_ite_conformal_mean,
            "epsilons": epsilons
        })
        df_eval.to_csv(f"../../results/outputs/r_learner/setup{setup_name}/eval_epsilon/simulations_{setup_name}_{str(n)}_{learner_name}_{adaptive_conformal_name}_eval.csv", index=False)

 36%|███▋      | 4/11 [11:05<19:31, 167.32s/it]